# Loading data

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
data = pd.read_csv('../input/breast-cancer-wisconsin-data/data.csv')

In [ ]:
data.head()

In [ ]:
data.isnull().sum()

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
data,test = train_test_split(data,test_size = 0.2)

In [ ]:
data,validation = train_test_split(data,test_size = 0.2)

In [ ]:
data.shape

In [ ]:
test.shape

In [ ]:
data = data.drop(columns = ['id'])

In [ ]:
data['diagnosis'].unique()

In [ ]:
labels = data['diagnosis']

In [ ]:
data.drop(columns = ['diagnosis'],inplace = True)

In [ ]:
data = data.iloc[:,0:29]

In [ ]:
data.head()

In [ ]:
daig = validation['diagnosis']
validation.drop(columns = ['id','diagnosis'],inplace=True)
validation.iloc[:,0:29]


In [ ]:
validation = validation.iloc[:,0:29]

# Data Visualiaztion

In [ ]:
data['area_mean'].hist(bins = 30)

In [ ]:
columns = data.columns

In [ ]:
columns

In [ ]:

j = 0
for i in columns:
  j = j+1
  plt.figure(figsize = (5,5))
  plt.title(i)
  plt.hist(data[i],30)



In [ ]:
# Most of the features right skewed , except symmetry mean, smoothness_mean and smoothness_worst

In [ ]:
plt.figure(figsize=(30,30))
sns.heatmap(data.corr(),annot=True)

In [ ]:

mean = ['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean',
       'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean']

In [ ]:
data_mean = data[mean]

In [ ]:
plt.figure(figsize=(20,20))
sns.heatmap(data_mean.corr(),annot=True)

In [ ]:
# High Positive Correlation between 
# 1. radius_mean and perimeter_mean
# 2. radius_mean and area_mean
# 3.radius_mean and concave_points_mean
# 4.perimeter_mean and area_mean
# 5.perimeter_mean and concave_points_mean
# 6.area_mean and concave_points_mean
# 7.compactness_mean and concavity_mean
# 8.compactness_mean and concave points_mean
# 9.concavity_mean and concave points_mean

In [ ]:
sns.pairplot(data_mean)


In [ ]:
se = ['radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',]
data_se = data[se]

In [ ]:
plt.figure(figsize=(20,20))
sns.heatmap(data_se.corr(),annot=True)

In [ ]:
# High Positive Correlation between 
# 1. radius_se and perimeter_se
# 2. radius_se and area_se
# 3.radius_se and concave_points_se
# 4.perimeter_se and area_se
# 5.compactness_se and concavity_se


In [ ]:
# sns.pairplot(data_se)

In [ ]:
worst = ['radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst']
data_worst = data[worst]

In [ ]:
plt.figure(figsize=(20,20))
sns.heatmap(data_worst.corr(),annot=True)

In [ ]:
# High Positive Correlation between 
# 1. radius_worst and perimeter_worst
# 2. radius_worst and area_worst
# 3.radius_worst and concave_points_worst
# 4.perimeter_worst and area_worst
# 5.perimeter_worst and concave_points_worst
# 6.area_worst and concave_points_worst
# 7.compactness_worst and concavity_worst
# 8.compactness_worst and concavity_points_worst
# 9.concavity_worst and concavity_points_worst

In [ ]:
np.std(data['concavity_mean'])

In [ ]:
np.std(data['compactness_mean'])

In [ ]:
np.std(data['concave points_mean'])

In [ ]:
print(np.std(data['concavity_se']))
print(np.std(data['compactness_se']))

In [ ]:
print(np.std(data['concavity_worst']))
print(np.std(data['compactness_worst']))
print(np.std(data['concave points_worst']))

In [ ]:
print(np.std(data['radius_mean']))
print(np.std(data['area_mean']))
print(np.std(data['perimeter_mean']))

In [ ]:
# Area is proportional to radius squared hence it has highest standard deviation 

In [ ]:

data_x = data

In [ ]:
data_x.shape

# Data Preparation for model training

In [ ]:
from sklearn.preprocessing import Normalizer

In [ ]:
n = Normalizer()

In [ ]:
data_x = n.fit_transform(data_x)

In [ ]:
import tensorflow.keras as k

In [ ]:
import tensorflow as tf


In [ ]:
# labels = tf.keras.utils.to_categorical(labels)

In [ ]:
map = {'M':1,'B':0}

In [ ]:
labels.value_counts()

In [ ]:
labels = labels.map(map
           )

In [ ]:
labels

In [ ]:
validation = n.transform(validation)

In [ ]:
daig = daig.map(map)

In [ ]:
test.head()


In [ ]:
test_y = test['diagnosis']

In [ ]:
test = test.drop(columns = ['id','diagnosis'])

In [ ]:
test= test.iloc[:,0:29]

In [ ]:
test.head()

In [ ]:
test = n.transform(test)

In [ ]:
test_y = test_y.map(map)

In [ ]:
test_y.head()

# Building and Training a Neural Network

In [ ]:
# The model hyperparameters are decided by rounds of calculated trail and error.
epoch = 600
model = k.models.Sequential([
                                  k.layers.Dense(12,input_dim =29,activation = 'relu'),
                                    k.layers.Dropout(0.5),
                                    k.layers.Dense(5,activation = 'relu'),
                                    k.layers.Dropout(0.5),
#                                     k.layers.Dense(5,activation = 'relu'),
#                                     k.layers.Dropout(0.5),
                                    k.layers.Dense(1,activation = 'sigmoid')
                                    
                                    

])
model_check = k.callbacks.ModelCheckpoint('model_check.h5',save_best_only=True)
model.compile(loss = ['binary_crossentropy'],optimizer ='adam',metrics = ['accuracy'])
history = model.fit(data_x,labels,epochs=epoch,verbose=0,validation_data = (validation,daig),callbacks = [model_check])

In [ ]:
model = k.models.load_model('model_check.h5')

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(1,epoch+1)

plt.figure(figsize = (30,30))
plt.subplot(2,1,1)
plt.plot(epochs_range,acc,label = 'training_acc')
plt.plot(epochs_range,val_acc,label = 'val_acc')
plt.legend(loc = 'lower right',fontsize = 30)
plt.title('Training and Validation Accuracy',fontsize = 30)
plt.subplot(2,1,2)
plt.plot(epochs_range,loss,label = 'training_loss')
plt.plot(epochs_range,val_loss,label = 'val_loss')
plt.legend(loc = 'lower right',fontsize = 30)
plt.title('Training and Validation Loss',fontsize = 30)

In [ ]:

predictions = model.predict(test)

In [ ]:
pred = []
for i in range(0,len(predictions)):
  if predictions[i][0]>=0.5:
    pred.append(1)
  else:
    pred.append(0)  

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y,pred)

In [ ]:
#Achieveing 94% accuracy

In [ ]:
predictions_train = model.predict(data_x)

In [ ]:
pred_train = []
for i in range(0,len(predictions_train)):
  if predictions_train[i][0]>=0.5:
    pred_train.append(1)
  else:
    pred_train.append(0)  

In [ ]:
accuracy_score(labels,pred_train) 

In [ ]:
# Lower Accuracy on training data, no Overfitting or under fitting of the model detected.